In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [3]:
# Read .csv file
df_dodge_cummins = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\dodge-cumminsR.csv")

<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_18364\4269422931.py:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
  df_dodge_cummins = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\dodge-cumminsR.csv")


In [4]:
# Check the columns
df_dodge_cummins.columns

Index(['ModTYPE', 'Hardware', 'CUMMINS P/N', 'CUMMINS E/C', 'Latest Software',
       'Older Software Versions', 'Model Year and Trans', 'Engine',
       'Description', 'Example VINs'],
      dtype='object')

In [5]:
# Get the index with empty rows
df_dodge_cummins_empty_rows_index = df_dodge_cummins[df_dodge_cummins['Hardware'].isna() == True].index

In [6]:
# Drop empty rows
df_dodge_cummins_empty_rows_dropped = df_dodge_cummins.drop(df_dodge_cummins_empty_rows_index)

In [7]:
# Get the indexes with the repeated columns
df_dodge_cummins_repeated_col_index = df_dodge_cummins_empty_rows_dropped[df_dodge_cummins_empty_rows_dropped['Hardware'] == 'Hardware'].index

In [8]:
# Drop the repeated cols
df_dodge_cummins_repeated_col_dropped = df_dodge_cummins_empty_rows_dropped.drop(df_dodge_cummins_repeated_col_index)

In [9]:
# Function to fill out null rows
def fill_null_rows(df):
    # Loop to iterate over all cols
    for col in df.columns:
        df[col] = df[col].fillna("Not Available")
    return df

In [10]:
# Function to remove the whitespaces
def remove_whitespaces(df):
    # Loop to iterate over all cols
    for col in df.columns:
        # Remove the writespaces
        df[col] = df[col].str.strip()

    return df

In [11]:
# Function to split multiple part numbers originally in the same line to one row each
def split_part_numbers(df, col):

    # Call the function to remove the whitespaces
    df_rm_spaces = remove_whitespaces(df)

    df_copy = df_rm_spaces.copy()

    # Create a list with the items in each row 
    df_copy[col] = df_copy[col].str.split(",")
    # Explode the items in different rows each and keep the info from the other rows
    df_splitted = df_copy.explode(col)
    # Return the df with the exploded part numbers
    return df_splitted

In [12]:
# Call the function to fill out the null values
df_dodge_cummins_fill_out_null_rows = fill_null_rows(df_dodge_cummins_repeated_col_dropped)

In [13]:
# Print the cols
df_dodge_cummins_repeated_col_dropped.columns

Index(['ModTYPE', 'Hardware', 'CUMMINS P/N', 'CUMMINS E/C', 'Latest Software',
       'Older Software Versions', 'Model Year and Trans', 'Engine',
       'Description', 'Example VINs'],
      dtype='object')

### Call the function to the explode the following cols:
- ModTYPE
- Hardware
- CUMMINS P/N
- CUMMINS E/C
- Latest Software
- Older Software Versions

In [14]:
df_dodge_cummins_expl_modtype_col = split_part_numbers(df_dodge_cummins_repeated_col_dropped, "ModTYPE")

In [15]:
df_dodge_cummins_expl_hdw_col = split_part_numbers(df_dodge_cummins_expl_modtype_col, "Hardware")

In [16]:
df_dodge_cummins_expl_cumm_pn_col = split_part_numbers(df_dodge_cummins_expl_hdw_col, "CUMMINS P/N")

In [17]:
df_dodge_cummins_expl_cumm_ec_col = split_part_numbers(df_dodge_cummins_expl_cumm_pn_col, "CUMMINS E/C")

In [18]:
df_dodge_cummins_expl_last_sftw_col = split_part_numbers(df_dodge_cummins_expl_cumm_ec_col, "Latest Software")

In [19]:
df_dodge_cummins_expl_old_sftw_col = split_part_numbers(df_dodge_cummins_expl_last_sftw_col, "Older Software Versions")

In [20]:
# Reset the index
df_dodge_cummins_reset_index = df_dodge_cummins_expl_old_sftw_col.reset_index(drop=True)

In [21]:
# Show the cols available
df_dodge_cummins_reset_index.columns

Index(['ModTYPE', 'Hardware', 'CUMMINS P/N', 'CUMMINS E/C', 'Latest Software',
       'Older Software Versions', 'Model Year and Trans', 'Engine',
       'Description', 'Example VINs'],
      dtype='object')

### Cols to keep:
- ModTYPE
- Hardware
- CUMMINS P/N
- CUMMINS E/C
- Latest Software
- Older Software Versions

In [22]:
# List with cols to keep
cols_to_keep = ["ModTYPE", "Hardware", "CUMMINS P/N", "CUMMINS E/C", "Latest Software", "Older Software Versions"]

In [23]:
# Create a df with needed cols
df_dodge_cummins_cols_to_keep = df_dodge_cummins_reset_index[cols_to_keep]

In [24]:
# Put the hdw col as the first one
df_dodge_cummins_hdw_col_first = df_dodge_cummins_cols_to_keep[[
    "Hardware",
    "ModTYPE", 
    "CUMMINS P/N", 
    "CUMMINS E/C", 
    "Latest Software", 
    "Older Software Versions"]]

In [25]:
# Check the info
df_dodge_cummins_hdw_col_first.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4387 entries, 0 to 4386
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Hardware                 4387 non-null   object
 1   ModTYPE                  4387 non-null   object
 2   CUMMINS P/N              4387 non-null   object
 3   CUMMINS E/C              4387 non-null   object
 4   Latest Software          4387 non-null   object
 5   Older Software Versions  4387 non-null   object
dtypes: object(6)
memory usage: 205.8+ KB


## Standardizing Part Number Dataset Columns
Each part number dataset contains columns with different names because they originate from different manufacturers. One of the goals of cleaning these datasets is to create a standard format that allows for the identification of the hardware part number when inputting other part numbers, such as software part numbers or system part numbers.

Since the identification of the hardware part number will be performed by inputting multiple part numbers at once (via a .csv file), it is necessary — except for the hardware column — to rename the other columns to a standard format: `pnNum` (e.g., `pn1`, `pn2`, etc.). This ensures consistency and enables automated matching and processing across datasets from various manufacturers.

In [26]:
def rename_col_names(df):
    '''
    Except the column hardware, the function standardize the names for pnNumber (pn1, pn2).
    Needed due to each part number dataset has different column names, so that 

    Parameters:
    df (DataFrame): main df.
    col_list (list): List of original column names.

    Returns
    df: Return the main df with the renamed cols

    '''

    # List with the col names
    col_name_list = list(df.columns) 
    
    # Dict to append the old names (key): new names (values) 
    new_cols_dict = {}

    # Count the items under the col list
    col_num = len(col_name_list)
    # Loop to iterate over the list
    for col in col_name_list:
        # Condition to define when lower the col name and when rename it
        if col == "Hardware":
            new_cols_dict[col] = col.lower()
        else:
            # Convert the string to number to sum with the stirng pn and
            # append as value to the dict
            new_cols_dict[col] = "pn" + str(col_num)
            # Return the number from string to int
            col_num = int(col_num)
        
        # Subtract one to create the next col name
        num_col = col_num - 1
        col_num = num_col

    # Return the main df with the cols remanes
    return df.rename(columns=new_cols_dict)

In [27]:
# Call the function to rename the cols
df_dodge_cummins_rn_cols = rename_col_names(df_dodge_cummins_hdw_col_first)

In [ ]:
# Export to .csv file
df_dodge_cummins_hdw_col_first.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\data_cleaned\pn-cummins.csv", index=False)

In [30]:
def import_datasets_to_db(df, df_name=str):
    
    '''
    Import the df cleaned to the db on postgreSQL

    Parameters: 
        df (DataFrame): main df to import to the postgreSQL db.
        df_name: string to label the df into the db.

    Returns: 
        Import the df to the db.
    '''
    # Setting up the connection with the PostgreSQL
    dbname="prescreen_diag_data_api"
    user="postgres"
    password="shakey-10"
    host="localhost"
    port="5432"

    # String connection for SQLAlchemy (using psycopg2 as driver)
    engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}")

    return df.to_sql(df_name, engine, if_exists='replace', index=False)

In [31]:
# Call the function to import the df cleaned to the postgreSQLdb
import_datasets_to_db(df_dodge_cummins_rn_cols, 'part_numbers_cummins')

387